<a href="https://colab.research.google.com/github/hoso1000/kaken-kubun-finder/blob/main/kaken_kubun_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 科研費区分ファインダー（仮）

応募しようとする科研費の課題名＋概要をもとに、すでに採択されている科研費課題（KAKENデータベースに掲載されているもの）から課題名・概要が似ている課題を探し出し、似ている課題がどの小区分で多く採択されているかチェックできます。

In [5]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 必要なライブラリ等をインストール
!pip install -q transformers==4.7.0 fugashi ipadic
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.996.5
!pip install transformers

/bin/bash: aptitude: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import BertJapaneseTokenizer, BertModel
import torch

# 日本語Sentence-BERTクラス定義
class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)

In [7]:
# 日本語モデルの読み込み
model = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens-v2")

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/667 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [12]:
# KAKEN DB採択課題データを読み込み、データフレームkakenに格納
import pandas as pd
path = '/content/drive/MyDrive/kaken-kubun-finder/'
# KAKEN DB採択課題はkaken.nii.ac.jp_BCW.csvとして上記フォルダにダウンロードしておく
# （このフォルダ内のkaken.nii.ac.jp_BCW.csvは2018-2022年度開始の基盤B・C・若手の全課題を2022年10月にDLしたもの）
kaken = pd.read_csv(path + 'kaken.nii.ac.jp_BCW.csv') 

# kakenの研究課題名・概要が空欄の課題を削除
kaken = kaken[(kaken['研究課題名'].isna()==False)&(kaken['研究開始時の研究の概要'].isna()==False)].reset_index(drop = True)

# 英文の課題を削除し、インデックスを振り直す
l_drop = []
for i in kaken.index:
  kadai_i = kaken.iat[i, 1]
  gaiyo_i = kaken.iat[i, 27]
  if len((kadai_i).encode())/len(kadai_i) < 1.5:
    l_drop.append(i)
  elif len((gaiyo_i).encode())/len(gaiyo_i) < 1.5:
    l_drop.append(i)
  else: continue

kaken = kaken.drop(kaken.index[l_drop]).reset_index(drop = True)
print('KAKENデータ読み込み完了 件数：',len(kaken))

KAKENデータ読み込み完了 件数： 88259


採択課題の課題名の文ベクトルとしてsentence-vector-v2.ptを作成するときはこのタイミングで以下のコードを実行：
import re
sentences = []
for i in kaken.index:
  gaiyo = kaken.at[i, '研究開始時の研究の概要']
  gaiyo = gaiyo + '：' + kaken.at[i,'研究課題名']
  sentences.append(gaiyo)

sentencesの文ベクトルを計算してファイル保存
sentence_vectors = model.encode(sentences)
torch.save(sentence_vectors, path + 'sentence-vector-v2.pt')

In [13]:
# 文ベクトルをすでにsentence-vector-v2.ptに保存済みの場合このセルで読み出す
sentence_vectors = torch.load(path + 'sentence-vector-v2.pt')

In [19]:
# 近い区分を調べたい課題の研究課題名を入力する
qkadai = input('研究課題名を入力: ')
# 近い区分を調べたい課題の研究概要を入力する
qgaiyo = input('研究概要を入力: ')
# 最も似ている課題名から何件取り出すかclosest_nに入力する、指定がなければ10件
n = input('何件取り出す？半角数字で入力：')
try:
    closest_n = int(n)
except ValueError:
    closest_n = 10

研究課題名を入力: がん免疫療法のT細胞機能評価を可能とする3次元がんチップの開発と実証
研究概要を入力: 本研究では申請者が開発に関わってきたがん模倣デバイス技術を応用することで 1)血管網を介したT細胞とがん組織との遠隔での相互作用 2)がん組織が形成する微小環境における近距離での細胞間相互作用の2点が評価可能な、新しいタイプの Tumoron-a-chip モデルの構築 に取り組む。この in vitroの評価モデルと光学的観察系を構築することで、T細胞ががん微小環境内の細胞へ与える免疫反応を評価可能にすることを目指す。このような、生体内を模倣した評価モデルが構築可能となれば、これまで明らかにできなかった事象にアプローチすることができ、有効性の高いT細胞製剤開発技術の飛躍的向上につながる。
何件取り出す？半角数字で入力：


In [20]:
import re
# 課題名について文ベクトルを計算し、query_embeddingsに格納する
query = re.sub('．','。',qgaiyo)
query = query.split('。')
query.append(qkadai)
# 概要について文ベクトルを計算し、query_embeddingsに格納する
query_embeddings = model.encode(query).numpy()

In [21]:
import scipy.spatial

# 各データを格納するリストを作成する
r_query = [] #応募課題名
r_result1 = [] #似ている課題
r_result2 =[] #似ている課題の審査区分
r_distance = [] #コサイン距離

# 応募課題の文ベクトルquery_embeddingsと採択課題の文ベクトルsentence_vectorsとのコサイン距離を求め、distancesに格納する
distances = scipy.spatial.distance.cdist(query_embeddings, sentence_vectors, metric="cosine")[0]

# コサイン距離distancesについて小さい順にソートして辞書resultsに格納
results = zip(range(len(distances)), distances)
results = sorted(results, key=lambda x: x[1])

print(qkadai + '\n==========\n')
# resultsが小さかったもの上位closest_n位について
for idx, distance in results[0:closest_n]:
   # 課題名が似ている課題名、採択課題と応募課題とのコサイン距離、似ている課題の審査区分, 研究の概要を格納する
    try:
      r_query.append(qkadai) # 応募課題
      r_distance.append(distance / 2) #コサイン距離
      r_result1.append(kaken.iat[idx,1]) #似ている課題
      r_result2.append(kaken.iat[idx,14]) #似ている課題の審査区分
      print(kaken.iat[idx,1], "(Score: %.4f)" % (distance / 2), kaken.iat[idx,14])
    except Exception as e:
      print(e)
    else: continue

がん免疫療法のT細胞機能評価を可能とする3次元がんチップの開発と実証

がん免疫療法のT細胞機能評価を可能とする３次元がんチップの開発と実証 (Score: 0.0433) 小区分90110:生体医工学関連
がん免疫療法の評価を可能とする3次元がんモデルの構築 (Score: 0.0998) 小区分90110:生体医工学関連
高速な３次元組織化を実現する細胞パッチ法を利用した革新的ヒト腫瘍モデルの創製 (Score: 0.1019) 小区分90110:生体医工学関連
外部刺激に応答するがん標的セラノスティックナノデバイスの開発 (Score: 0.1019) 小区分52040:放射線科学関連
人為的がん組織構築による膵癌-間質相互作用の解明と制御 (Score: 0.1028) 小区分55010:外科学一般および小児外科学関連
バイオフィジックスの論理的分子デザインに基づいた安全なCAR-T細胞の開発 (Score: 0.1093) 小区分52050:胎児医学および小児成育学関連
新しいCancer-on-chipの腫瘍ダイナミクス解析による創薬支援 (Score: 0.1101) 小区分90110:生体医工学関連
癌オルガノイドを用いた口腔扁平上皮癌幹細胞の治療標的分子の探索 (Score: 0.1119) 小区分57020:病態系口腔科学関連
NK細胞サブセットによるクローン病新規治療標的の抽出 (Score: 0.1123) 小区分53010:消化器内科学関連
血液適合性高分子材料における癌細胞の認識性と運動能の制御メカニズムの解析 (Score: 0.1142) 小区分90120:生体材料学関連


In [22]:
# 結果をcsvファイルとしてresultフォルダ内に保存する
import datetime
dt_now = datetime.datetime.now()

# 結果格納用のDataFrame df_rを作成する
df_r = pd.DataFrame()
df_r['研究課題名(和文)'] = r_query
df_r['似ている課題名'] = r_result1
df_r['審査区分'] = r_result2
df_r['コサイン距離'] = r_distance

df_r.to_csv(path + 'result/kaken-kubun-finder-result-' + dt_now.strftime('%Y-%m-%d_%H-%M-%S')  + '.csv', encoding = 'cp932')